In [39]:
pip install geopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 4.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [38]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium
from folium.plugins import MarkerCluster
from folium import plugins
from folium import IFrame
from IPython.display import IFrame
import time
import re

In [2]:
# 발급받은 API 키
api_key = "tXsTu3r9EslMT0tkioVB"
api_secret = 'qXR7d0OGoT'

In [3]:
# API 호출 URL
url = "https://openapi.naver.com/v1/search/news.json"

In [4]:
# 검색할 키워드들
keywords = ["강도", "살인", "강간", "성폭행", "칼부림", "묻지마", "폭행", "음주운전"]

In [5]:
# API 호출 시 사용할 헤더
headers = {
    "X-Naver-Client-Id": api_key,
    "X-Naver-Client-Secret": api_secret
}

In [6]:
# 결과를 저장할 리스트
news_list = []
naver_news_list = []

In [7]:
# 키워드별로 검색 결과 가져오기
for keyword in keywords:
    params = {
        "query": keyword,
        "display": 20,  # 가져올 결과 개수
        "start": 1      # 시작 위치 (1부터)
    }
    
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        for item in data['items']:
            link = item['link']
            if link.startswith("https://n.news.naver.com"):
                try:
                    news_response = requests.get(link)
                    if news_response.status_code == 200:
                        soup = BeautifulSoup(news_response.content, "html.parser")
                        title_element = soup.find("h2", id="title_area") or soup.find("h2", class_="end_tit")
                        title = title_element.get_text() if title_element else ""

                        content_element = soup.find("div", id="newsct_article") or soup.find("div", id="articeBody")
                        content = content_element.get_text() if content_element else ""

                        news_dict = {
                            "keyword": keyword,
                            "title": title,
                            "content": content
                        }
                        
                        naver_news_list.append(news_dict)
                    else:
                        print(f"뉴스 링크 '{link}' 접속 실패")
                except Exception as e:
                    print(f"뉴스 링크 '{link}' 크롤링 실패:", e)
            time.sleep(0.2)  # 0.2초 쉬기
    else:
        print(f"검색어 '{keyword}'의 API 호출 실패")

In [8]:
naver_news_list

[{'keyword': '강도',
  'title': "'발암물질' 아스파탐 걱정하면서… 술·담배는 그대로 하시나요?",
  'content': '\n\n\n\n\n\n사진=연합뉴스지난달 세계보건기구 산하 국제암연구소와 국제식량농업기구·세계보건기구 합동 식품첨가물전문가위원회가 아스파탐을 발암물질 2B군으로 분류했다. 이후 전 세계적으로 아스파탐에 대한 우려와 논쟁이 끊이지 않고 있다.아스파탐은 백색의 결정성 분말로 일반 설탕보다 200배의 단맛을 내는 인공감미료다. 지난 1974년 미국 식품의약청에서 식품첨가물로 분류돼 사용이 승인된 이후, 다이어트 음료, 젤리, 아이스크림, 요구르트 등 유제품은 물론이고 시리얼, 치약, 츄어블 비타민과 같은 다양한 제품에 광범위하게 사용돼 왔다. 우리나라에서도 1980년부터 탄산음료나 소주, 막걸리 등에 첨가해 왔으며, 강한 단맛을 내면서도 칼로리가 거의 없는 특징으로 인해 최근에는 저칼로리 대체감미료로 비만을 걱정하는 많은 이들로부터 각광을 받고 있다.국제암연구소에서 말하는 ‘발암물질 2B군’은 무엇을 의미하는 것일까? 국제암연구소는 지난 1969년부터 발암물질 및 요인에 대해 평가해 오고 있다. 2023년 현재까지 1110종에 대해 평가를 마치고 위험도에 따라 4개 군(1군, 2A군, 2B군, 3군)으로 분류해 알리고 있다.1군은 담배처럼 인체에서의 발암성이 확실한 것을 말한다. 2A군은 발암성이 확실치는 않으나 상당히 의심되는 수준을 말한다. 2B군은 발암 가능성이 낮지만, 없다고 할 수는 없으니 관심을 가지고 지켜볼 것을 주문하는 수준이다.아스파탐을 매일 마시던 사람들이 공포에 떨고 있는데, 알고 보면 2B군인 아스파탐이 암을 일으킨다는 근거는 적다. 담배가 발암물질 덩어리라는 것을 모르는 사람은 없지만, 사람들이 흔히 마시는 술도 구강암, 인두암, 후두암, 식도암, 대장암, 간암, 유방암을 일으키는 1군 발암물질이고, 소시지, 햄, 베이컨 같은 가공육도 대장암을 일으키는 1군 발암물질이다.우리가 매우 자주 먹는 소고

In [9]:
geolocator = Nominatim(user_agent="myGeocoder")

In [10]:
# 뉴스에서 추출한 정보를 저장할 리스트
crime_location = []

In [11]:
# naver_news_list에서 뉴스 제목과 내용 추출
for news in naver_news_list:
    news_text = news["title"] + " " + news["content"]
    
    # 지역 키워드 추출
    location_keywords = re.findall(r"\b(서울|부산|대구|인천|광주|대전|울산|충남|천안|아산)\b", news_text)
    
    if location_keywords:
        for location_keyword in location_keywords:
            location = geolocator.geocode(location_keyword)
            if location:
                crime_info = {
                    "사건": news['keyword'],
                    "지역": location_keyword,
                    "위도": location.latitude,
                    "경도": location.longitude
                }
                crime_location.append(crime_info)
            
            else:
                print(f"키워드 '{keyword}'로 추정되는 사건의 지역 정보를 찾을 수 없음")

In [13]:
crime_location

[{'사건': '강도', '지역': '서울', '위도': 37.5666791, '경도': 126.9782914},
 {'사건': '강도', '지역': '대전', '위도': 36.3497007, '경도': 127.3849016},
 {'사건': '강도', '지역': '대전', '위도': 36.3497007, '경도': 127.3849016},
 {'사건': '강도', '지역': '광주', '위도': 35.1594647, '경도': 126.8515034},
 {'사건': '강도', '지역': '광주', '위도': 35.1594647, '경도': 126.8515034},
 {'사건': '강도', '지역': '대전', '위도': 36.3497007, '경도': 127.3849016},
 {'사건': '살인', '지역': '서울', '위도': 37.5666791, '경도': 126.9782914},
 {'사건': '살인', '지역': '서울', '위도': 37.5666791, '경도': 126.9782914},
 {'사건': '살인', '지역': '서울', '위도': 37.5666791, '경도': 126.9782914},
 {'사건': '살인', '지역': '서울', '위도': 37.5666791, '경도': 126.9782914},
 {'사건': '살인', '지역': '서울', '위도': 37.5666791, '경도': 126.9782914},
 {'사건': '살인', '지역': '서울', '위도': 37.5666791, '경도': 126.9782914},
 {'사건': '살인', '지역': '서울', '위도': 37.5666791, '경도': 126.9782914},
 {'사건': '살인', '지역': '인천', '위도': 37.456, '경도': 126.7052},
 {'사건': '살인', '지역': '서울', '위도': 37.5666791, '경도': 126.9782914},
 {'사건': '살인', '지역': '서울', '위도': 37.5666791, '경도

In [14]:
# 데이터를 DataFrame으로 변환
crime_df = pd.DataFrame(crime_location)

In [15]:
# '지역'을 기준으로 피봇 테이블 생성
pivot_table = crime_df.pivot_table(index='지역', columns='사건', aggfunc='size', fill_value=0)

In [16]:
# 피봇 테이블을 출력해봅니다.
print(pivot_table)

사건  강도  묻지마  살인  성폭행  음주운전  칼부림  폭행
지역                                 
광주   2    0   0    0     0    0   0
대전   3    0   0    0     0    0   1
부산   0    1   0    0     8    0   1
서울   1   15  27    9     1   24   1
울산   0    0   0    0     2    0   0
인천   0    1   3    0     0    2   0


In [73]:
# 이전에 생성한 m 객체가 있다면, 그 객체를 사용하고 없다면 새로 생성
if 'm' not in globals():
    m = folium.Map(location=[37.5665, 126.9780], zoom_start=12)

In [74]:
# 맵 중심 위치 설정
map_center = [37.5666791, 126.9782914]
m = folium.Map(location=map_center, zoom_start=10)

In [75]:
# 사건별 색상 설정
colors = {'강도': 'red', '살인': 'black', '강간': 'yellow', '성폭행': 'blue', "칼부림": 'green', "묻지마": 'purple', "폭행": 'orange', "음주운전": 'pink'}

In [76]:
# Create a marker cluster
marker_cluster = plugins.MarkerCluster().add_to(m)

In [77]:
# 피봇 테이블을 이용하여 지도에 마크 표시
for location, row in pivot_table.iterrows():
    total_crime_count = row.sum()  # 해당 지역의 총 사건 갯수
    
    # 같은 지역에 1개 이상의 사건이 발생한 경우
    if total_crime_count > 1:
        # 지역의 모든 사건을 순회하면서 마크를 추가
        for crime_type, count in row.items():
            if count > 0:
                folium.CircleMarker(
                    location=[crime_df.loc[crime_df['지역'] == location, '위도'].iloc[0], 
                              crime_df.loc[crime_df['지역'] == location, '경도'].iloc[0]],
                    radius=count * 5,  # 마크 크기 설정
                    color=colors[crime_type],
                    fill=True,
                    fill_color=colors[crime_type],
                    fill_opacity=0.7,
                    popup=f"{crime_type} 사건\n지역: {location}\n사건 수: {count}",
                    tooltip=str(count)  # 마크 내부에 사건 숫자를 표시할 툴팁 설정
                ).add_to(marker_cluster)  # 클러스터에 마커 추가
    else:
        # 같은 지역에 1개의 사건만 발생한 경우
        most_common_crime = row.idxmax()
        count = row.max()
        folium.CircleMarker(
            location=[crime_df.loc[crime_df['지역'] == location, '위도'].iloc[0], 
                      crime_df.loc[crime_df['지역'] == location, '경도'].iloc[0]],
            radius=count * 5,  # 마크 크기 설정
            color=colors[most_common_crime],
            fill=True,
            fill_color=colors[most_common_crime],
            fill_opacity=0.7,
            popup=f"{most_common_crime} 사건\n지역: {location}\n사건 수: {count}",
            tooltip=str(count)  # 마크 내부에 사건 숫자를 표시할 툴팁 설정
        ).add_to(marker_cluster)  # 클러스터에 마커 추가


In [78]:
# Add a legend to the map with adjusted styling
legend_html = """
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 100px; height: 200px;
     border:2px solid grey; z-index:9999; font-size: 5px; /* 글자 크기 조절 */
     background-color:white;
     padding: 5px;
     text-align: center;
     ">   
     <p><i class="fa fa-circle fa-2x" style="color:red"></i> 강도</p>
     <p><i class="fa fa-circle fa-2x" style="color:black"></i> 살인</p>
     <p><i class="fa fa-circle fa-2x" style="color:yellow"></i> 강간</p>
     <p><i class="fa fa-circle fa-2x" style="color:blue"></i> 성폭행</p>
     <p><i class="fa fa-circle fa-2x" style="color:green"></i> 칼부림</p>
     <p><i class="fa fa-circle fa-2x" style="color:purple"></i> 묻지마</p>
     <p><i class="fa fa-circle fa-2x" style="color:orange"></i> 폭행</p>
     <p><i class="fa fa-circle fa-2x" style="color:pink"></i> 음주운전</p>
</div>
"""

# Apply the adjusted legend style to the map
m.get_root().html.add_child(folium.Element(legend_html))

# Display the map
m


In [79]:
# 지도를 HTML 파일로 저장
m.save("crime1_map.html")